In [2]:
import numpy as np
import pandas as pd
#from haversine import haversine, Unit
import geopy.distance

In [3]:
small_df = pd.read_csv('../DAOU/test_small_out_v1.csv', index_col=0, encoding='euc-kr')
small_df['네이버 이런점이 좋아요 {항목:좋아요수}'].notnull
small_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 610 entries, 0 to 0
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   사업장명                    610 non-null    object 
 1   업소유형                    251 non-null    object 
 2   관광숙박업상세명                319 non-null    object 
 3   건물용도명                   209 non-null    object 
 4   지번주소                    610 non-null    object 
 5   도로명주소                   602 non-null    object 
 6   위도                      583 non-null    float64
 7   경도                      583 non-null    float64
 8   별점                      172 non-null    float64
 9   방문자 리뷰수                 231 non-null    object 
 10  블로그 리뷰수                 229 non-null    object 
 11  지하철역과의 거리               177 non-null    object 
 12  도보시간                    37 non-null     object 
 13  객실수                     307 non-null    float64
 14  시설면적                    358 non-null    floa

In [4]:
large_df = pd.read_csv('../DAOU/test_large_out_v1.csv', index_col=0, encoding='euc-kr')
large_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6664 entries, 0 to 0
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   사업장명                    6664 non-null   object 
 1   업소유형                    874 non-null    object 
 2   업태구분명                   6664 non-null   object 
 3   지번주소                    6653 non-null   object 
 4   도로명주소                   3986 non-null   object 
 5   위도                      5287 non-null   float64
 6   경도                      5287 non-null   float64
 7   별점                      310 non-null    float64
 8   방문자 리뷰수                 685 non-null    object 
 9   블로그 리뷰수                 513 non-null    object 
 10  지하철역과의 거리               774 non-null    object 
 11  도보시간                    94 non-null     object 
 12  한실수                     5434 non-null   float64
 13  양실수                     5347 non-null   float64
 14  욕실수                     4813 non-null   flo

In [5]:
subway = pd.read_csv('../DABA/서울시 역사마스터 정보.csv', index_col=0, encoding='euc-kr')
subway

,역사명,호선,위도,경도
역사_ID,,,,
9996,미사,5호선,127.193877,37.560927
9995,강일,5호선,127.175930,37.557490
4929,김포공항,김포골드라인,126.801868,37.562360
4928,고촌,김포골드라인,126.770345,37.601243
4927,풍무,김포골드라인,126.732387,37.612488
...,...,...,...,...
154,종로5가,1호선,127.001849,37.570926
153,종로3가,1호선,126.991847,37.570406
152,종각,1호선,126.982923,37.570161


In [6]:
small_df.iloc[0]['사업장명']

print(np.nan)

nan


In [20]:
# .. 가까운 지하철 거리 및 도보 시간 구하기
# 가까운 지하철 구하기

small_dist = []

for i in range(small_df.shape[0]):
#for i in range(1):
    dummy = []

    if np.isnan(small_df.iloc[i]['위도']) :

        dummy.append(np.nan)
        find_min_j = np.nan
        find_stn_nam = np.nan
        find_stn_lat = np.nan
        find_stn_lon = np.nan
        nearest_stn = np.nan
        time_to_near = np.nan

    else:

        for j in range(subway.shape[0]):

            hotel_latlon = (small_df.iloc[i]['위도'], small_df.iloc[i]['경도'])
            subway_latlon = (subway.iloc[j]['경도'], subway.iloc[j]['위도'])    # 원본데이터에서 위경도 컬럼명 바뀌어 있음

            dist = geopy.distance.geodesic(hotel_latlon, subway_latlon).m
            dummy.append(dist)

        find_min_j = np.argmin(dummy)
        dist_to_stn = np.nanmin(dummy)
        time_to_stn = np.nanmin(dummy)/66.7

        find_stn_nam = subway.iloc[find_min_j]['역사명']
        find_stn_lat = subway.iloc[find_min_j]['위도']
        find_stn_lon = subway.iloc[find_min_j]['경도']

        
            
        #print(f"호텔:{small_df.iloc[i]['사업장명']} {small_df.iloc[i]['지번주소']} {min(dummy)}m")
    small_dist.append([small_df.iloc[i]['사업장명'],
                        small_df.iloc[i]['지번주소'],
                        small_df.iloc[i]['위도'],
                        small_df.iloc[i]['경도'],
                        find_stn_nam,
                        find_stn_lat,
                        find_stn_lon,
                        dist_to_stn,
                        time_to_stn]) # 시속 4km --> 분속 66.7m



In [23]:
find_small_dist = pd.DataFrame(small_dist, columns=['사업장명','지번주소','호텔 위도', '호텔 경도','역이름', '역 위도','역 경도', '거리','도보시간(분)'])

#for i in range(10):
#    print(f"호텔:{small_df.iloc[i]['사업장명']}, {df.iloc[i]['거리']}")
find_small_dist.info()
find_small_dist

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   사업장명     610 non-null    object 
 1   지번주소     610 non-null    object 
 2   호텔 위도    583 non-null    float64
 3   호텔 경도    583 non-null    float64
 4   역이름      583 non-null    object 
 5   역 위도     583 non-null    float64
 6   역 경도     583 non-null    float64
 7   거리       610 non-null    float64
 8   도보시간(분)  610 non-null    float64
dtypes: float64(6), object(3)
memory usage: 43.0+ KB


,사업장명,지번주소,호텔 위도,호텔 경도,역이름,역 위도,역 경도,거리,도보시간(분)
0,호텔 데님(Hotel Denim),서울특별시 강남구 개포동 1229-14,37.476190,127.046616,양재시민의숲(매헌),127.038420,37.470023,997.016642,14.947776
1,티마크그랜드호텔명동,서울특별시 중구 회현동1가 194-15,37.558454,126.978646,회현(남대문시장),126.978246,37.558514,35.993514,0.539633
2,위너스,서울특별시 금천구 독산동 335-21,37.471959,126.889817,독산,126.889249,37.466613,595.471106,8.927603
3,데이즈 호텔 명동,서울특별시 중구 충무로1가 24-30,37.560953,126.983631,명동,126.986325,37.560989,238.049587,3.568959
4,강남패밀리호텔,서울특별시 강남구 논현동 205-8,37.506474,127.030156,언주,127.033868,37.507287,340.365147,5.102926
...,...,...,...,...,...,...,...,...,...
605,하모니관광호텔,서울특별시 강동구 길동 387-1번지,37.538231,127.138108,길동,127.140004,37.537801,174.229810,2.612141
606,호텔 감(HOTEL GAM),서울특별시 강동구 천호동 452-30번지,37.538400,127.128669,강동,127.132481,37.535804,443.301249,6.646196
607,토요코인 서울 동대문 Ⅱ,서울특별시 중구 광희동2가 14-1,37.564521,127.007868,동대문역사문화공원,127.007885,37.565133,67.907433,1.018102
608,발리(BALI),서울특별시 강동구 성내동 417-5,37.529051,127.134907,둔촌동,127.136248,37.527788,183.596511,2.752571


In [24]:
small_df['가까운 지하철역'] = find_small_dist['역이름']
small_df['지하철역과의 거리'] = find_small_dist['거리']
small_df['도보시간'] = find_small_dist['도보시간(분)']

In [26]:
small_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 610 entries, 0 to 0
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   사업장명                    610 non-null    object 
 1   업소유형                    251 non-null    object 
 2   관광숙박업상세명                319 non-null    object 
 3   건물용도명                   209 non-null    object 
 4   지번주소                    610 non-null    object 
 5   도로명주소                   602 non-null    object 
 6   위도                      583 non-null    float64
 7   경도                      583 non-null    float64
 8   별점                      172 non-null    float64
 9   방문자 리뷰수                 231 non-null    object 
 10  블로그 리뷰수                 229 non-null    object 
 11  지하철역과의 거리               610 non-null    float64
 12  도보시간                    610 non-null    float64
 13  객실수                     307 non-null    float64
 14  시설면적                    358 non-null    floa

In [ ]:
colum_names2 = ['사업장명','업소유형','관광숙박업상세명','건물용도명','지번주소','도로명주소','가까운 지하철역', '지하철역과의 거리', '도보시간',
                '위도','경도', '별점','방문자 리뷰수', '블로그 리뷰수', '객실수','시설면적','시설규모','구비시설',
                '네이버 이런점이 좋아요 총합', '네이버 이런점이 좋아요 {항목:좋아요수}', '데이터갱신일자',
                '인허가일자','인허가취소일자','영업상태코드','폐업일자','휴업시작일자','휴업종료일자','재개업일자']

small_df = small_df[colum_names2]
small_df

In [ ]:
output_path = "../DAOU/test_small_out_v2.csv"
small_df.to_csv(output_path, encoding='euc-kr')